## PromptTemplate（字符串模板）

In [ ]:
from langchain_core.prompts import PromptTemplate

# 创建模板
template = PromptTemplate.from_template(
    "翻译成{target_lang}：{text}\n结果："
)

# 基本使用
result = template.invoke({
    "target_lang": "中文",
    "text": "Hello World"
})

print("输出：", result)

# 输出：翻译成中文：Hello World\n结果：

# 批量处理
results = template.batch([
    {"target_lang": "中文", "text": "Good morning"},
    {"target_lang": "日文", "text": "Thank you"}
])

print("批量输出：", results)

# 批量输出： [StringPromptValue(text='翻译成中文：Good morning\n结果：'), StringPromptValue(text='翻译成日文：Thank you\n结果：')]

输出： text='翻译成中文：Hello World\n结果：'
批量输出： [StringPromptValue(text='翻译成中文：Good morning\n结果：'), StringPromptValue(text='翻译成日文：Thank you\n结果：')]


## ChatPromptTemplate（聊天模板）

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# 创建聊天模板
template = ChatPromptTemplate.from_messages([
    ("system", "你是一个{role}专家"),
    ("human", "请解释：{question}")
])

# 单个调用
prompt_value = template.invoke({
    "role": "数学",
    "question": "什么是微积分？"
})

print("Messages:", prompt_value.to_messages())

# 管道到模型
chain = template | model
response = chain.invoke({
    "role": "数学", 
    "question": "什么是微积分？"
})

print("回复：", response.content)


Messages: [SystemMessage(content='你是一个数学专家', additional_kwargs={}, response_metadata={}), HumanMessage(content='请解释：什么是微积分？', additional_kwargs={}, response_metadata={})]
回复： 微积分是数学的一个重要分支，主要研究变化和运动的概念。它将两大主要部分结合在一起，即微分和积分。

1. **微分**：微分主要关注函数的变化率。通过微分，我们可以找到函数在某一点的瞬时变化率，即导数。比如，物体的速度就是位置随时间变化的导数。

2. **积分**：积分则与累积量有关，主要用于求和和计算面积等问题。积分的结果可以看作是某一函数在一个区间内的“总变化量”。例如，计算一个曲线下方的面积可以通过对该曲线对应的函数进行积分来实现。

微积分的基本定理连接了微分和积分，表明微分和积分是互为反操作的。这一理论在物理、工程、经济学、生物学等多个领域有着广泛的应用，帮助我们理解和描述现实世界中的变化和关系。


## MessagesPlaceholder

In [ ]:

from datetime import datetime
from langchain_core.messages import AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是OpenAI开发的聊天机器人，请根据用户的提问进行回复，当前的时间为:{now}"),
    # 有时候可能还有其他的消息，但是不确定
    MessagesPlaceholder("chat_history"),
    HumanMessagePromptTemplate.from_template("请讲一个关于{subject}的冷笑话"),
]).partial(now=datetime.now())

# 补充 chat_history 和 subject 参数
chat_prompt = chat_prompt.invoke({
    "chat_history": [
        ("human", "我叫慕小课"),
        AIMessage("你好，我是ChatGPT，有什么可以帮到您"),
    ],
    "subject": "程序员",
})

print(chat_prompt.to_string())

System: 你是OpenAI开发的聊天机器人，请根据用户的提问进行回复，当前的时间为:2025-12-06 23:32:37.930508
Human: 我叫慕小课
AI: 你好，我是ChatGPT，有什么可以帮到您
Human: 请讲一个关于程序员的冷笑话


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019af44b-29ab-7581-9e68-ac73f4ca322c,id=019af44b-29ab-7581-9e68-ac73f4ca322c; trace=019af44b-29ab-7581-9e68-ac73f4ca322c,id=019af44b-29ab-7581-9e68-ac73f4ca322c


### 字符串拼接

In [ ]:
from langchain_core.prompts import PromptTemplate

# PromptTemplate 拼接 str
prompt = (
        PromptTemplate.from_template("请讲一个关于{subject}的冷笑话")
        + ",让我开心下" +
        "\n使用{language}语言"
)

prompt = prompt.invoke({"subject": "程序员", "language": "中文"})

print(prompt.to_string())

请讲一个关于程序员的冷笑话,让我开心下
使用中文语言


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019af448-dd21-7d71-b9b3-fc0b5a0aa502,id=019af448-dd21-7d71-b9b3-fc0b5a0aa502; trace=019af448-dd21-7d71-b9b3-fc0b5a0aa502,id=019af448-dd21-7d71-b9b3-fc0b5a0aa502


### 消息模板拼接

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是OpenAI开发的聊天机器人，请根据用户的提问进行回复，我叫{username}"),
])
human_chat_prompt = ChatPromptTemplate.from_messages([
    ("human", "{query}")
])

# 拼接两个 ChatPromptTemplate
chat_prompt = system_chat_prompt + human_chat_prompt

print(chat_prompt.invoke({
    "username": "慕小课",
    "query": "你好，你是?"
}))


messages=[SystemMessage(content='你是OpenAI开发的聊天机器人，请根据用户的提问进行回复，我叫慕小课', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，你是?', additional_kwargs={}, response_metadata={})]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019af44a-8113-7282-a328-5f995117e053,id=019af44a-8113-7282-a328-5f995117e053; trace=019af44a-8113-7282-a328-5f995117e053,id=019af44a-8113-7282-a328-5f995117e053


### 提示模板嵌套拼接

**PipelinePromptTemplate（废弃）**：在LangChain 0.1.0版本被弃用，并在0.2.0版本完全移除。

In [25]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel

# 复现各个子模板
instruction_prompt = PromptTemplate.from_template("你正在模拟{person}")
example_prompt = PromptTemplate.from_template(
    """下面是一个交互例子：

Q: {example_q}
A: {example_a}"""
)
start_prompt = PromptTemplate.from_template(
    """现在，你是一个真实的人，请回答用户的问题:

Q: {input}
A:"""
)

# 主模板
full_template = PromptTemplate.from_template("""{instruction}

{example}

{start}""")

# 使用 RunnableParallel 复现 pipeline 效果
pipeline_prompt = RunnableParallel(
    instruction=instruction_prompt,
    example=example_prompt, 
    start=start_prompt
) | full_template

# 使用
result = pipeline_prompt.invoke({
    "person": "雷军",
    "example_q": "你最喜欢的汽车是什么?",
    "example_a": "小米su7",
    "input": "你最喜欢的手机是什么?"
})
print(result.to_string())

text='你正在模拟雷军'

text='下面是一个交互例子：\n\nQ: 你最喜欢的汽车是什么?\nA: 小米su7'

text='现在，你是一个真实的人，请回答用户的问题:\n\nQ: 你最喜欢的手机是什么?\nA:'


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-283b-7fe3-9d67-86d1b659fe6d; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2840-76b3-9b91-7915032fed15; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2841-7452-a679-fcfdaefb9b7e; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2841-7452-a679-fcfdaefb9b7e; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2843-7d23-9c2f-96c43708292d; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2843-7d23-9c2f-96c43708292d; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2844-7152-9fe2-ad7b29c4c71f; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id=019af45b-2844-7152-9fe2-ad7b29c4c71f; trace=019af45b-283b-7fe3-9d67-86d1b659fe6d,id